# Reading OSM data

Reading OSM data can be configured. You will find example config in the configs directory. Available as a jupyter notebook or wiki page.

## Setting up the config

OSM data contains wast amount of information, to restrict what is being read, you can adjust the tags for OSM nodes and links that make it through

```yml
OSM_TAGS:
  # tags that are included when reading OSM data
  USEFUL_TAGS_NODE:
    - ref
    - highway
  USEFUL_TAGS_PATH:
    - bridge
    - tunnel
    - oneway
    - lanes
    - highway
    - service
    - access
    - railway
    - route
    - lanes
    - oneway
    - psv
    - vehicle
    - junction
    - osmid
```

GeNet will only read the ways in OSM with tags defined in `MODES:  MODE_INDICATORS:`---you need to define the mode for specific OSM way tags, GeNet will read only the network edges for which it understands in terms of which mode of transport can move on it. For example in the `slim_config.yml`:

```yml
MODES:
  MODE_INDICATORS:
    # These refer to tags stored in OSM, the mode assignment to tags is manual and definitely not exhaustive. 
    # If a tag is not listed here, it will not be read into the network. This is where you should define how dense
    # you want your network to be.
    highway:
      motorway:
        - car
      motorway_link:
        - car
      trunk:
        - car
      trunk_link:
        - car
      primary:
        - car
        - walk
        - bike
      primary_link:
        - car
        - walk
        - bike
      secondary:
        - car
        - walk
        - bike
      secondary_link:
        - car
        - walk
        - bike
      tertiary:
        - car
        - walk
        - bike
      tertiary_link:
        - car
        - walk
        - bike
      unclassified:
        - car
        - walk
        - bike
```

GeNet has pre-defined default capacity and speed values for a number of OSM tags. These can be found in [genet/outputs_handler/matsim_xml_values.py](https://github.com/arup-group/genet/blob/master/genet/outputs_handler/matsim_xml_values.py#L1). In the config for reading OSM, you can decide which tag a particular mode should default to if values for it do not exist.

```yml
  DEFAULT_OSM_TAG_VALUE:
  # GeNet will first look at OSM tags to infer the matsim values the link should have. For the types and values head over to 
  # outputs_handler/matsim_xml_values.py.
    car: secondary
    bus: secondary
    rail: railway
    subway: railway
    bike: living_street
    walk: pedestrian
```

There are currently two caveats:
- The number of lanes in a link, i.e. `permlanes` is taken from OSM data if present (and read by config 
under `OSM_TAGS: USEFUL_TAGS_PATH` and `OSM_TAGS: PUMA_GRAPH_TAGS`) and integer convertible (e.g. not `'2;3'`)
- `capacity` given in `matsim_xml_values.py` is for a single lane, the output network will feature a `capacity` value 
which is the result of `permlanes * capacity`, where the latter `capacity` is the base lane capacity defined in `matsim_xml_values.py`

## Reading OSM

GeNet ingests OSM data with .osm or .osm.pbf extensions, which can be obtained from

- http://download.geofabrik.de/ for well-defined regions/countries
- through JOSM for small subsets and saved to the aforementioned extensions

GeNet creates a Multi Directed Graph from OSM data, meaning that there can be more than one edge between the same pair of nodes directed from `origin_node` to `destination_node`, if that is the case within the OSM file.

In [1]:
from genet import read_osm

We use the `read_osm` method pointing at the osm file we want to read and a config defining which OSM tags to read and which modes to assign to the resulting `Network` links. For details on how to set up the config head over to the [wiki page](https://github.com/arup-group/genet/wiki/Usage:-2.2.-Reading-Data:-OSM#setting-up-the-config).

GeNet ingests OSM data with .osm or .osm.pbf extensions, which can be obtained from

- http://download.geofabrik.de/ for well-defined regions/countries
- through JOSM for small subsets and saved to the aforementioned extensions

In [2]:
n = read_osm(
    "../example_data/example.osm",
    "../genet/configs/OSM/slim_config.yml",
    num_processes=2,
    epsg="epsg:27700",
)

2022-07-14 15:27:55,774 - Building OSM graph from file ../example_data/example.osm
2022-07-14 15:27:56,540 - Creating networkx graph from OSM data
2022-07-14 15:27:56,541 - OSM: Extract Nodes and Paths from OSM data
2022-07-14 15:27:56,898 - OSM: Add each OSM way (aka, path) to the OSM graph
2022-07-14 15:27:56,899 - 1 or 2 batches of size 64 ish
2022-07-14 15:27:56,925 - Created OSM edges
2022-07-14 15:27:56,943 - 1 or 2 batches of size 4348 ish
2022-07-14 15:27:58,563 - Added 8695 nodes
2022-07-14 15:27:58,566 - 2 or 3 batches of size 401 ish
2022-07-14 15:28:03,352 - Generated 802 link ids.
2022-07-14 15:28:03,577 - Added 802 links
2022-07-14 15:28:03,579 - Deleting isolated nodes which have no edges.
2022-07-14 15:28:03,989 - Removed 8132 nodes.


In [3]:
n.print()

Graph info: Name: Network graph
Type: MultiDiGraph
Number of nodes: 563
Number of edges: 802
Average in degree:   1.4245
Average out degree:   1.4245 
Schedule info: Schedule:
Number of services: 0
Number of routes: 0
Number of stops: 0


In [4]:
# n.plot()

In [5]:
n.node_attribute_summary(data=False)

attribute
├── id
├── x
├── y
├── lat
├── lon
└── s2_id


In [6]:
n.link_attribute_summary(data=False)

attribute
├── permlanes
├── freespeed
├── capacity
├── oneway
├── modes
├── from
├── to
├── s2_from
├── s2_to
├── length
├── attributes
│   ├── osm:way:highway
│   ├── osm:way:osmid
│   ├── osm:way:lanes
│   ├── osm:way:psv
│   ├── osm:way:tunnel
│   ├── osm:way:railway
│   └── osm:way:access
└── id


In [7]:
n.link("1")

{'permlanes': 1.0,
 'freespeed': 22.22,
 'capacity': 1500.0,
 'oneway': '1',
 'modes': ['bike', 'car', 'walk'],
 'from': '107319',
 'to': '2440641461',
 's2_from': 5221390693512237449,
 's2_to': 5221390693522407815,
 'length': 7.300429822404873,
 'attributes': {'osm:way:highway': 'primary', 'osm:way:osmid': 24865179},
 'id': '1'}

#### Remark:
A graph generated from OSM is most likely not **strongly connected** which is something that MATSim expects for the mode `car` and if using `multimodal.contrib` also for modes `walk` and `bike`.